# Enhance MagIC file for Shaat dike data

Two MagIC files are generated from project research:
- The MagIC file exported from Demag_GUI associated with the thermal demagnetization data
- The MagIC file exported from the IRM database associated with the rock magnetic experiments

These files need to be merged together into a single contribution as was done in:
https://github.com/Swanson-Hysell-Group/2021_ECMB/blob/main/data/rockmag_pmag_MagIC/combine_MagIC.ipynb

## Import Python packages

In [1]:
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
import pandas as pd
pd.set_option("display.max_columns", None)

## Import paleomagnetic data

The data that have been analyzed using demag_gui with MagIC files being saved out of demag_gui are imported below.

In [2]:
pmag_dir_path = '../data/pmag/demag'
pmag_contribution = cb.Contribution(pmag_dir_path)
pmag_measurements = pmag_contribution.tables['measurements'].df
pmag_specimens = pmag_contribution.tables['specimens'].df
pmag_samples = pmag_contribution.tables['samples'].df
pmag_sites = pmag_contribution.tables['sites'].df
pmag_locations = pmag_contribution.tables['locations'].df

-I- Using online data model
-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.org


## Enhance the site location precision and add site metadata

An issue with the CIT data format is that the precision on the site longitude/latitude is low given the ASCII file format. We therefore need to bring in more precise locations which are within the site_locations.csv file and add them to the site table.

Additionally, MagIC requires that sites have additional metadata including:
- geologic_classes
- geologic_types
- lithologies
- age (or age_high and age_low)
- age_unit
  
These data are also in the site_locations.csv file and can be merged into the sites table.

In [3]:
# get more precise site locations
SD_site_locations = pd.read_csv('../data/field_data/site_locations.csv', sep=',', header=0, index_col='site')
# map site locations to site data
pmag_sites['lat'] = SD_site_locations['latitude'][pmag_sites['site']].tolist()
pmag_sites['lon'] = SD_site_locations['longitude'][pmag_sites['site']].tolist()
pmag_sites['geologic_classes'] = SD_site_locations['geologic_classes'][pmag_sites['site']].tolist()
pmag_sites['geologic_types'] = SD_site_locations['geologic_types'][pmag_sites['site']].tolist()
pmag_sites['lithologies'] = SD_site_locations['lithologies'][pmag_sites['site']].tolist()
pmag_sites['age_high'] = SD_site_locations['age_high'][pmag_sites['site']].tolist()
pmag_sites['age_low'] = SD_site_locations['age_low'][pmag_sites['site']].tolist()
pmag_sites['age_unit'] = SD_site_locations['age_unit'][pmag_sites['site']].tolist()

### Filter out tilt corrected sites when not in Leger panel

We only have structural control in the Leger panel. We exported both geographic and tilt-corrected coordinates for every site out of Pmag_GUI, but should only report those for the Leger panel. We can find the sites within the Leger panel and only keep pmag_sites['dir_tilt_correction'] of 100 for those sites.

In [4]:
leger_sites = SD_site_locations[SD_site_locations['locale'].str.contains("Leger", na=False)].index
leger_sites

Index(['SD36', 'SD37', 'SD38', 'SD39', 'SD40', 'SD42', 'SD43', 'SD44', 'SD45',
       'SD46', 'SD47', 'SD48', 'SD49', 'SD50', 'SD51', 'SD52', 'SD53', 'SD54',
       'SD55', 'SD56', 'SD57', 'SD58', 'SD59', 'SD60', 'SD61', 'SD62', 'SD63',
       'SD64', 'SD65', 'SD66', 'SD67', 'SD68', 'SD69', 'SD70', 'SD71', 'SD72',
       'SD73', 'SD74', 'SD75', 'SD76', 'SD77', 'SD78', 'SD79', 'SD80', 'SD86',
       'SD87', 'SD88', 'SD89', 'SD90', 'SD91', 'SD92', 'SD93', 'SD94', 'SD95',
       'SD96', 'SD97', 'SD98', 'SD99', 'S100', 'S101', 'S102', 'S103', 'S104',
       'S105', 'S106', 'S107'],
      dtype='object', name='site')

In [5]:
pmag_sites = pmag_sites[~((pmag_sites['dir_tilt_correction'] == 100) & (~pmag_sites['site'].isin(leger_sites)))]

## Add necessary metadata to the locations table

The following information needs to be provided for the location:
- location_type
- geologic_classes
- lithologies
- age * or age_low and age_high*
- age_unit

In [6]:
unique_geologic_classes = pmag_sites['geologic_classes'].dropna().unique()
location_geologic_classes = ": ".join(map(str, unique_geologic_classes))

unique_lithologies = pmag_sites['lithologies'].dropna().unique()
location_lithologies = ": ".join(map(str, unique_lithologies))

highest_age_high = pmag_sites['age_high'].max()
lowest_age_low = pmag_sites['age_low'].min()

In [7]:
pmag_locations['location_type'] = 'Region'
pmag_locations['geologic_classes'] = location_geologic_classes
pmag_locations['lithologies'] = location_lithologies
pmag_locations['age_low'] = lowest_age_low
pmag_locations['age_high'] = highest_age_high
pmag_locations['age_unit'] = 'Ma'
pmag_locations

,analysts,citations,lat_n,lat_s,location,lon_e,lon_w,location_type,geologic_classes,lithologies,age_low,age_high,age_unit
location name,,,,,,,,,,,,,
Zalawt plain,NaN,This study,17.1,17.1,Zalawt plain,54.9,54.9,Region,Sedimentary: Igneous: Metamorphic,Diamictite: Felsic Dike: Mafic Dike: amphiboli...,660,815,Ma


In [8]:
pmag_filtered_dir = '../data/pmag'
pmag_contribution.tables['locations'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "locations.txt")
pmag_contribution.tables['sites'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "sites.txt")
pmag_contribution.tables['samples'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "samples.txt")
pmag_contribution.tables['specimens'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "specimens.txt")
pmag_contribution.tables['measurements'].write_magic_file(dir_path=pmag_filtered_dir,custom_name = "measurements.txt")

-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt
-I- 1 records written to locations file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/sites.txt
-I- 207 records written to sites file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt
-I- 386 records written to samples file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt
-I- 1435 records written to specimens file
-I- overwriting /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/measurements.txt
-I- 9099 records written to measurements file


'/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/measurements.txt'

In [9]:
ipmag.upload_magic(dir_path=pmag_filtered_dir, input_dir_path=pmag_filtered_dir)

-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt file successfully read in
1  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/locations.txt written to  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt file successfully read in
386  records written to file  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/samples.txt written to  /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/upload.txt
-I- /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt file successfully read in
-I- dropping these columns: location, site from the /Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/specimens.txt table
1435  records written to fil

('/Users/unimos/0000_Github/2025_Oman_Paleogeography/data/pmag/Zalawt-plain_11.Mar.2025_3.txt',
 {'status': True,
  'validation': {'errors': [{'table': 'locations',
     'column': 'lithologies',
     'message': 'The locations table column "lithologies" value "Felsic Dike" is not in the "Lithologies" controlled vocabulary.',
     'rows': [1]},
    {'table': 'sites',
     'column': 'method_codes',
     'message': 'The sites table is missing required column "method_codes".',
     'rows': [6, 10, 37, 152, 164, 167, 206]},
    {'table': 'sites',
     'column': 'citations',
     'message': 'The sites table is missing required column "citations".',
     'rows': [6, 10, 37, 152, 164, 167, 206]},
    {'table': 'sites',
     'column': 'lithologies',
     'message': 'The sites table column "lithologies" value "Felsic Dike" is not in the "Lithologies" controlled vocabulary.',
     'rows': [4,
      95,
      96,
      107,
      128,
      129,
      167,
      173,
      174,
      175,
      178